In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score 

In [2]:
# Load datasets
train_df = pd.read_csv("data/train.csv")
test_df  = pd.read_csv("data/test.csv")

# Drop columns with no variance (from EDA)
drop_cols = ['X11','X93','X107','X233','X235','X268','X289','X290','X293','X297','X330','X347']
train_df = train_df.drop(columns=drop_cols, errors="ignore")
test_df  = test_df.drop(columns=drop_cols, errors="ignore")

# 3) One-Hot Encode categorical columns
categorical_cols = ['X0','X1','X2','X3','X4','X5','X6','X8']
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

# Ohe on train categoricals
ohe_train = ohe.fit_transform(train_df[categorical_cols])
# Ohe on test categoricals
ohe_test  = ohe.transform(test_df[categorical_cols])

# Transformed dataframes (drop original categoricals, join OHE)
train_transformed = train_df.drop(columns=categorical_cols).join(ohe_train)
test_transformed  = test_df.drop(columns=categorical_cols).join(ohe_test)

# Quick checks
print("Shapes")
print("Original train_df shape:    ", train_df.shape)
print("Transformed train_df shape: ", train_transformed.shape)

print("\n Dropped Categorical Columns")
print(categorical_cols)

print("\n New OHE Columns Names")
print(ohe.get_feature_names_out(categorical_cols)[:20])  # show first 20 OHE columns
print(f"Total new OHE columns: {len(ohe.get_feature_names_out(categorical_cols))}")

Shapes
Original train_df shape:     (4209, 366)
Transformed train_df shape:  (4209, 553)

 Dropped Categorical Columns
['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

 New OHE Columns Names
['X0_a' 'X0_aa' 'X0_ab' 'X0_ac' 'X0_ad' 'X0_af' 'X0_ai' 'X0_aj' 'X0_ak'
 'X0_al' 'X0_am' 'X0_ao' 'X0_ap' 'X0_aq' 'X0_as' 'X0_at' 'X0_au' 'X0_aw'
 'X0_ax' 'X0_ay']
Total new OHE columns: 195


In [3]:
# review from EDA: find the most correlated features 
num_cols = train_transformed.select_dtypes(include=np.number).columns
corr_series = (
    train_transformed[num_cols]
    .corr(numeric_only=True)['y']
    .drop(labels=['y','ID'], errors='ignore')
    .dropna()
    .abs()
    .sort_values(ascending=False)
)
print(corr_series.index[:12])  # shows top dozen names in order

Index(['X314', 'X261', 'X127', 'X29', 'X232', 'X279', 'X263', 'X0_az', 'X76',
       'X54', 'X136', 'X328'],
      dtype='object')


# GRID SEARCH

### Random Forest + Grid Search

In [18]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# ----------------------------
# 1. Load Data
# ----------------------------
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

# Columns to drop
drop_cols = ['X4','X11','X93','X107','X233','X235','X268',
             'X289','X290','X293','X297','X330','X347']

train = train.drop(columns=drop_cols, errors="ignore")
test = test.drop(columns=drop_cols, errors="ignore")

# Separate features & target
y = train["y"]
X = train.drop(["y", "ID"], axis=1)
X_test_final = test.drop(["ID"], axis=1)

# ----------------------------
# 2. Preprocessing
# ----------------------------
categorical_features = ["X0", "X1", "X2", "X3", "X5", "X6", "X8"]
numeric_features = [col for col in X.columns if col not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

# ----------------------------
# 3. Pipeline with Random Forest
# ----------------------------
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", rf)
])

# ----------------------------
# 4. Hyperparameter Grid
# ----------------------------
param_grid = {
    "model__n_estimators": [100, 200, 500],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5, 10],
    "model__max_features": ["log2", "sqrt"]
}

# ----------------------------
# 5. Grid Search CV
# ----------------------------
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    error_score='raise',
    scoring="r2",
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X, y)

# ----------------------------
# 6. Results
# ----------------------------
print("✅ Best Parameters:", grid_search.best_params_)
print("✅ Best CV R²:", grid_search.best_score_)

# ----------------------------
# 7. Evaluate on Hold-out Split
# ----------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)

print("\nValidation Performance:")
print("MSE:", mean_squared_error(y_val, y_val_pred))
print("R²:", r2_score(y_val, y_val_pred))

# ----------------------------
# 8. Predictions on Test Set
# ----------------------------
test_preds = best_model.predict(X_test_final)
submission = pd.DataFrame({"ID": test["ID"], "y": test_preds})

submission.to_csv("submission.csv", index=False)
print("\n🎉 submission.csv saved with predictions!")

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categori

[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=100; total time=   0.6s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=100; total time=   0.6s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=100; total time=   0.6s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=100; total time=   0.6s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=100; total time=   0.6s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=200; total time=   0.6s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, model__n_estimators=200; total time=   0.7s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=2, mode

/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=5, model__n_estimators=100; total time=   0.5s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=5, model__n_estimators=100; total time=   0.4s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=5, model__n_estimators=100; total time=   0.4s
[CV] END model__max_depth=None, model__max_features=log2, model__min_samples_split=5, model__n_estimators=100; total time=   0.7s


...

/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_split=5, model__n_estimators=500; total time=   1.8s


/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 4] during transform. These unknown categori

[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_split=10, model__n_estimators=500; total time=   1.0s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_split=10, model__n_estimators=500; total time=   1.0s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_split=10, model__n_estimators=500; total time=   1.0s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_split=10, model__n_estimators=500; total time=   1.0s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_split=10, model__n_estimators=500; total time=   0.8s
✅ Best Parameters: {'model__max_depth': 20, 'model__max_features': 'sqrt', 'model__min_samples_split': 10, 'model__n_estimators': 500}
✅ Best CV R²: 0.548964552693593

Validation Performance:
MSE: 42.60398360226967
R²: 0.7262833271918707

🎉 submission.csv saved with predictions!


/Users/raulguzman/Desktop/AIPM/mercedes/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


### again but only w/ selected features

In [19]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score

# ----------------------------
# 1. Load Data
# ----------------------------
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

# Target
y = train["y"]

# Selected numeric features
selected_features = ['X314', 'X261', 'X127', 'X29', 'X232',
                     'X279', 'X263', 'X76', 'X54', 'X136', 'X328']

# ----------------------------
# 2. One-Hot Encode X0 and keep only X0_az
# ----------------------------
ohe = OneHotEncoder(drop=None, sparse_output=False, handle_unknown="ignore")

# Fit OHE on X0 from training set
X0_encoded = ohe.fit_transform(train[["X0"]])
X0_encoded_df = pd.DataFrame(X0_encoded, columns=ohe.get_feature_names_out(["X0"]))

# Keep only X0_az (if it exists in categories)
if "X0_az" in X0_encoded_df.columns:
    train["X0_az"] = X0_encoded_df["X0_az"].values
else:
    train["X0_az"] = 0  # fallback in case category not present

# Do the same for test set
X0_encoded_test = ohe.transform(test[["X0"]])
X0_encoded_test_df = pd.DataFrame(X0_encoded_test, columns=ohe.get_feature_names_out(["X0"]))

if "X0_az" in X0_encoded_test_df.columns:
    test["X0_az"] = X0_encoded_test_df["X0_az"].values
else:
    test["X0_az"] = 0

# ----------------------------
# 3. Build Final Feature Sets
# ----------------------------
X = train[selected_features + ["X0_az"]]
X_test_final = test[selected_features + ["X0_az"]]

# ----------------------------
# 4. Model & Grid Search
# ----------------------------
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid = {
    "n_estimators": [100, 200, 500],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "max_features": ["log2", "sqrt"]
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    error_score='raise',
    scoring="r2",
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X, y)

# ----------------------------
# 5. Results
# ----------------------------
print("✅ Best Parameters:", grid_search.best_params_)
print("✅ Best CV R²:", grid_search.best_score_)

# ----------------------------
# 6. Hold-out Validation
# ----------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)

print("\nValidation Performance:")
print("MSE:", mean_squared_error(y_val, y_val_pred))
print("R²:", r2_score(y_val, y_val_pred))

# ----------------------------
# 7. Predictions on Test Set
# ----------------------------
test_preds = best_model.predict(X_test_final)
submission = pd.DataFrame({"ID": test["ID"], "y": test_preds})
submission.to_csv("submission.csv", index=False)
print("\n🎉 submission.csv saved with predictions!")

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=N

## Regularization

In [13]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score

# Example with Ridge
ridge = Ridge(alpha=1.0, random_state=42)
scores = cross_val_score(ridge, X, y, cv=5, scoring="r2")

print("Ridge R² (CV mean):", scores.mean())

# Example with Lasso (feature selection)
lasso = Lasso(alpha=0.001, random_state=42, max_iter=5000)
scores = cross_val_score(lasso, X, y, cv=5, scoring="r2")

print("Lasso R² (CV mean):", scores.mean())

Ridge R² (CV mean): 0.5112569953174076
Lasso R² (CV mean): 0.5157719038481955


In [15]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# ----------------------------
# 1. Load Data
# ----------------------------
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

y = train["y"]

# Selected numeric features
selected_features = ['X314', 'X261', 'X127', 'X29', 'X232',
                     'X279', 'X263', 'X76', 'X54', 'X136', 'X328']

# ----------------------------
# 2. One-Hot Encode X0 and keep only X0_az
# ----------------------------
ohe = OneHotEncoder(drop=None, sparse_output=False, handle_unknown="ignore")

X0_encoded = ohe.fit_transform(train[["X0"]])
X0_encoded_df = pd.DataFrame(X0_encoded, columns=ohe.get_feature_names_out(["X0"]))

# Add X0_az (or 0 if missing)
train["X0_az"] = X0_encoded_df["X0_az"] if "X0_az" in X0_encoded_df.columns else 0

# Do the same for test
X0_encoded_test = ohe.transform(test[["X0"]])
X0_encoded_test_df = pd.DataFrame(X0_encoded_test, columns=ohe.get_feature_names_out(["X0"]))
test["X0_az"] = X0_encoded_test_df["X0_az"] if "X0_az" in X0_encoded_test_df.columns else 0

# Final feature sets
X = train[selected_features + ["X0_az"]]
X_test_final = test[selected_features + ["X0_az"]]

# ----------------------------
# 3. Split Train/Validation
# ----------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------------
# 4. Define Models
# ----------------------------
models = {
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.001, random_state=42, max_iter=5000),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42, max_iter=5000)
}

# ----------------------------
# 5. Train & Evaluate
# ----------------------------
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    
    cv_scores = cross_val_score(model, X, y, cv=5, scoring="r2")
    
    results[name] = {
        "Validation MSE": mse,
        "Validation R²": r2,
        "CV Mean R²": cv_scores.mean(),
        "CV Std": cv_scores.std()
    }
    
    print(f"\n📌 {name} Results")
    print("Validation MSE:", mse)
    print("Validation R²:", r2)
    print("CV Mean R²:", cv_scores.mean(), "±", cv_scores.std())
    
    # Feature coefficients
    coef_df = pd.DataFrame({
        "Feature": X.columns,
        "Coefficient": model.coef_
    }).sort_values(by="Coefficient", ascending=False)
    
    print("\nTop 5 Positive Coefficients:")
    print(coef_df.head(5))
    print("\nTop 5 Negative Coefficients:")
    print(coef_df.tail(5))

# ----------------------------
# 6. Pick Best Model & Predict Test
# ----------------------------
best_model_name = max(results, key=lambda x: results[x]["CV Mean R²"])
best_model = models[best_model_name]

print(f"\n✅ Best Model Selected: {best_model_name}")

best_model.fit(X, y)
test_preds = best_model.predict(X_test_final)

submission = pd.DataFrame({"ID": test["ID"], "y": test_preds})
submission.to_csv("submission_regularized.csv", index=False)
print("\n🎉 submission_regularized.csv saved with predictions!")


📌 Ridge Results
Validation MSE: 67.60293789260989
Validation R²: 0.565673214862612
CV Mean R²: 0.5112569953174076 ± 0.06713847587885641

Top 5 Positive Coefficients:
   Feature  Coefficient
6     X263    16.046353
8      X54    10.138275
7      X76    10.138275
10    X328    -0.656875
1     X261    -1.161884

Top 5 Negative Coefficients:
  Feature  Coefficient
9    X136   -10.138275
5    X279   -16.046353
4    X232   -16.046353
3     X29   -16.046353
2    X127   -19.095291

📌 Lasso Results
Validation MSE: 68.47660053751117
Validation R²: 0.5600602178585856
CV Mean R²: 0.5157719038481955 ± 0.07133695793813442

Top 5 Positive Coefficients:
   Feature   Coefficient
7      X76  4.972862e+01
11   X0_az  1.865655e-01
8      X54  1.146341e-11
6     X263  1.919616e-16
10    X328  0.000000e+00

Top 5 Negative Coefficients:
  Feature   Coefficient
4    X232 -7.912769e-12
1    X261 -1.122951e+00
0    X314 -2.890656e+00
2    X127 -1.918901e+01
3     X29 -8.559363e+01

📌 ElasticNet Results
Validat